## Loading the LLM

In [1]:
from dotenv import load_dotenv
from typing import TypedDict, List, Union

# Lang Graph Imports
from langgraph.graph import StateGraph, START, END

# Langchin Imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage

from aws_llm import llm

### Basic AI Agent 1

In [2]:
class AgentState(TypedDict):
    messages: List[HumanMessage]


def process(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    print(response.content)
    return state


graph = StateGraph(AgentState)
graph.add_node("process_node", process)
graph.add_edge(START, "process_node")
graph.add_edge("process_node", END)
agent = graph.compile()

user_input = input("Enter your message: ")
agent.invoke({"messages": [HumanMessage(content=user_input)]})

Hello! How are you doing today? Is there anything I can help you with?


{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]}

### Agent 2 (Context)

In [ ]:
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]


def process(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    print(response.content)
    state["messages"].append(AIMessage(content=response.content))
    print(f"Current State Messages:{state['messages']}")
    return state


graph2 = StateGraph(AgentState)
graph2.add_node("process_node", process)
graph2.add_edge(START, "process_node")
graph2.add_edge("process_node", END)
agent = graph2.compile()
user_input = input("Enter your message: ")
conversation_history = []
while user_input.lower() != "exit":
    conversation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({"messages": conversation_history})
    # Update conversation_history with the AI response
    conversation_history = result["messages"]

    # Remove oldest pair if more than 3 pairs (6 messages)
    if len(conversation_history) > 6:
        conversation_history = conversation_history[
            2:
        ]  # Remove first pair (2 messages)

    user_input = input("Enter your message: ")

Hi there! How can I help you today?
Current State Messages:[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={})]
There isn't a single "best" country in the world, as it depends entirely on what criteria you prioritize. What one person considers the best, another might not.

However, if you're looking for countries that frequently rank highly in various global indices for quality of life, happiness, economic stability, and social progress, some commonly cited examples include:

*   **Switzerland**
*   **Canada**
*   **Norway**
*   **Sweden**
*   **New Zealand**
*   **Finland**
*   **Denmark**
*   **Australia**
*   **Netherlands**
*   **Iceland**

Again, this is just a selection based on common rankings, and the "best" is subjective.
Current State Messages:[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! How can I he